In [1]:
import joblib
import pandas as pd
from tokenizers import (
    models,
    normalizers,
    pre_tokenizers,
    trainers,
    Tokenizer,
)

from datasets import Dataset
from tqdm.auto import tqdm
from transformers import PreTrainedTokenizerFast

In [2]:
def dummy(text):
    return text
test=pd.read_csv('test.csv')
vectorizer = joblib.load('vectorizer.pkl')
model = joblib.load('ensemble_SenPiece_LGBM.pkl')

In [3]:
LOWERCASE = False
VOCAB_SIZE = 228293
raw_tokenizer = Tokenizer(models.BPE(unk_token="[UNK]"))
raw_tokenizer.normalizer = normalizers.Sequence([normalizers.NFC()] + [normalizers.Lowercase()] if LOWERCASE else [])
raw_tokenizer.pre_tokenizer = pre_tokenizers.ByteLevel()
special_tokens = ["[UNK]", "[PAD]", "[CLS]", "[SEP]", "[MASK]"]
trainer = trainers.BpeTrainer(vocab_size=VOCAB_SIZE, special_tokens=special_tokens)
dataset = Dataset.from_pandas(test[['text']])
def train_corp_iter(): 
    for i in range(0, len(dataset), 1000):
        yield dataset[i : i + 1000]["text"]
raw_tokenizer.train_from_iterator(train_corp_iter(), trainer=trainer)
tokenizer = PreTrainedTokenizerFast(
    tokenizer_object=raw_tokenizer,
    unk_token="[UNK]",
    pad_token="[PAD]",
    cls_token="[CLS]",
    sep_token="[SEP]",
    mask_token="[MASK]",
)
tokenized_texts_test = []

for text in tqdm(test['text'].tolist()):
    tokenized_texts_test.append(tokenizer.tokenize(text))
    
tf_test = vectorizer.transform(tokenized_texts_test)

  0%|          | 0/1912 [00:00<?, ?it/s]

In [4]:
pred = model.predict(tf_test)
test['is_rumor'] = pred


In [5]:
with open('submission.csv', 'w', encoding='utf-8') as f:
    f.write('id\tlabel\n')
    for _, row in test.iterrows():
        if row['text'] == '':
            f.write(f"{row['mid']}\t0\n")
        else:
            f.write(f"{row['mid']}\t{row['is_rumor']}\n")

In [6]:
test

,mid,text,is_rumor
0,2dKrhwqWBVm_1195031270,蛔虫 美女 问世 一种 减肥 方法 不用 运动 不用 节食 吃 蛔虫 卵 服用 蛔虫 卵后 ...,1
1,2dKrzuu3skL_1644395354,传说 中 麦当劳 全国 通用 无线 上网 密码 真的 真的 假 还 伐 存 次 都 找 不到...,1
2,2dKs1KOT8t0_1314608344,顶花 黄瓜 猫腻 生长 剂 含 避孕药 青岛 新闻网 顶 鲜艳 黄花 黄瓜 市场 上 受欢迎...,1
3,2dKSSoZX2Sd_1300442112,临沂 市委书记 奸淫 上百 幼女 燕赵 都市网 原文 出处 人渣 无语 怒 抓狂 求证 真的...,1
4,2kiroqdeC1F_1764669250,更 亲近 人 发火 讨厌 人 亲近 人 大部分 时间 人 发火 是因为 亲近 人 更 期望 ...,0
...,...,...,...
1907,zzabOkjrC_1264341240,刚录 完 节目 评论 里 都 讨论 穿 平底鞋 穿 平底鞋 是因为 不想 小朋友 高太多 真...,0
1908,zzBmWlKN0_1725725780,自然 堂 全国 路演 随手 拍 自然 堂舞林 争霸 冠军 喜欢 张傲 一路 走来 都 支持 ...,0
1909,zzFTHcFe1_1628595353,回龙观 赶紧 检查一下 家里 牙膏 黑色 条 赶紧 扔掉 买 牙膏 时 牙膏管 反面 底部 ...,1
1910,zzj3JohLU_2683002893,買 牙膏 請 留心 买 牙膏 时 牙膏管 反面 底部 颜色 条 才 颜色 条有 含义 尽量 ...,1
